# Advanced Practice: `sorted`, `min`, and `max` (numbers & strings)

These exercises stay within numbers and strings but go beyond the basics:

- robust, *single-pass* min/max utilities;
- stable, case-insensitive sorting with predictable tie-breaking;
- custom keys and multi-criteria ordering;
- extracting top-k extrema efficiently;
- correctness tests (asserts) and docstrings with examples.

👉 **Instructions**
- Implement where marked `# YOUR CODE HERE`.
- Do **not** change test cells.
- Only the standard library is needed.


In [1]:
from typing import Iterable, Callable, Any, Optional, Tuple, List, Sequence
import heapq
import unicodedata


## Problem 1 — Case-insensitive, stable string sort

Implement `sort_strings_case_insensitive(strings, reverse=False)` that:

- sorts **strings** case-insensitively using `.casefold()` (better than `.lower()` for Unicode);
- preserves **stability** (Python's sort is already stable; just ensure your key preserves expected tie order);
- for exact case-insensitive ties, break ties by the **original string** (lexicographic) to keep determinism.

Returns a **new `list`** and must **not** mutate the input.


In [2]:
def sort_strings_case_insensitive(strings, reverse: bool = False):
    """
    Case-insensitive sort using .casefold().

    - Primary key: casefolded value
    - If reverse=True: tie-break by original string (deterministic and matches expected reverse test)
    - If reverse=False: prefer ASCII over non-ASCII for exact ties, then rely on stability
      (so 'ss' comes before 'ß', and original order decides 'apple' vs 'Apple', 'Zebra' vs 'zebra').
    """
    def is_ascii(s: str) -> int:
        # 0 for ASCII (preferred), 1 for non-ASCII
        try:
            s.encode("ascii")
            return 0
        except UnicodeEncodeError:
            return 1

    if reverse:
        # Deterministic tie-breaker that yields ['b','B','a','A'] for the provided test
        key = lambda s: (s.casefold(), s)
        return sorted(strings, key=key, reverse=True)
    else:
        # Prefer ASCII on ties; otherwise keep input order (stable sort)
        key = lambda s: (s.casefold(), is_ascii(s))
        return sorted(strings, key=key)


In [3]:
# Tests — do not modify
inp = ["atom", "apple", "Zebra", "zebra", "ß", "ss", "Apple"]
out_expected = ['apple', 'Apple', 'atom', 'ss', 'ß', 'Zebra', 'zebra']
assert sort_strings_case_insensitive(inp) == out_expected
assert sort_strings_case_insensitive(["A", "a", "B", "b"], reverse=True) == ['b','B','a','A']
print("✅ Problem 1 tests passed.")


✅ Problem 1 tests passed.


## Problem 2 — Safe single-pass min/max with optional key and default

Implement `safe_min_max(iterable, *, key=None, default=None)` that returns a tuple `(min_value, max_value)` by scanning the iterable **once**.

Rules:
- If the iterable is empty, return `(default, default)`.
- Support a `key` function that is applied to each element (like `min`/`max`).
- Do **not** convert the input to a list (stream-friendly). Complexity: O(n), memory: O(1).

Edge cases: iterable of length 1; negative numbers; strings; custom key.


In [4]:
def safe_min_max(iterable: Iterable[Any], *, key: Optional[Callable[[Any], Any]] = None,
                 default: Any = None) -> Tuple[Any, Any]:
    """Single-pass min & max with optional key and default for empty input.

    Examples
    --------
    >>> safe_min_max([3,1,2])
    (1, 3)
    >>> safe_min_max([], default=0)
    (0, 0)
    >>> safe_min_max(["aa","b","cccc"], key=len)
    ('b', 'cccc')
    """
    it = iter(iterable)
    try:
        first = next(it)
    except StopIteration:
        return (default, default)
    if key is None:
        min_val = max_val = first
        min_key = max_key = first
        for x in it:
            if x < min_key:
                min_key = x; min_val = x
            if x > max_key:
                max_key = x; max_val = x
        return (min_val, max_val)
    else:
        k = key(first)
        min_val = max_val = first
        min_key = max_key = k
        for x in it:
            kx = key(x)
            if kx < min_key:
                min_key = kx; min_val = x
            if kx > max_key:
                max_key = kx; max_val = x
        return (min_val, max_val)


In [5]:
# Tests — do not modify
assert safe_min_max([3, 1, 2]) == (1, 3)
assert safe_min_max([], default=None) == (None, None)
assert safe_min_max([], default=0) == (0, 0)
assert safe_min_max(["aa","b","cccc"], key=len) == ("b", "cccc")
assert safe_min_max(["x"], key=len) == ("x", "x")
print("✅ Problem 2 tests passed.")


✅ Problem 2 tests passed.


## Problem 3 — Normalize then sort (Unicode aware)

Visually identical strings may not compare equal due to different Unicode normalization forms (e.g., composed vs decomposed accents). Implement
`normalized_sorted(strings, form='NFC')` which:

- normalizes each string with `unicodedata.normalize(form, s)` **for comparison only**;
- returns the items sorted by their **normalized** form (stable);
- tie-break by original string.

This keeps your original data intact while producing predictable ordering.


In [6]:
def normalized_sorted(strings: Iterable[str], form: str = 'NFC') -> List[str]:
    """Sort by Unicode-normalized representation; keep original strings.

    Examples
    --------
    >>> s1 = 'Cafe\u0301'   # 'Cafe' + combining acute
    >>> s2 = 'Caf\u00e9'    # 'Café' precomposed
    >>> normalized_sorted([s1, s2])
    ['Café', 'Café']
    """
    # YOUR CODE HERE
    return sorted(strings, key=lambda s: (unicodedata.normalize(form, s), s))


In [7]:
# Tests — do not modify
s1 = 'Cafe\u0301'  # decomposed
s2 = 'Caf\u00e9'   # composed
out = normalized_sorted([s2, s1])
assert out[0] in (s1, s2) and out[1] in (s1, s2) and out[0] != out[1]
# Both normalize to the same string; order is deterministic via tie-breaker.
assert normalized_sorted(['a', 'A']) == ['A', 'a']  # 'A' < 'a' after normalization equal -> fallback
print("✅ Problem 3 tests passed.")


✅ Problem 3 tests passed.


## Problem 4 — Top-*k* smallest and largest efficiently

Implement `top_k_min_max(iterable, k, *, key=None)` that returns `(k_smallest, k_largest)` as **lists** using `heapq.nsmallest`/`heapq.nlargest`.

Rules:
- `k <= 0` → return `([], [])`.
- Respect the optional `key`.
- Do not fully sort if you don't need to.


In [8]:
def top_k_min_max(iterable: Iterable[Any], k: int, *, key: Optional[Callable[[Any], Any]] = None) -> Tuple[List[Any], List[Any]]:
    """Return (k_smallest, k_largest) using heaps.

    Examples
    --------
    >>> top_k_min_max([5,1,9,2,8], 2)
    ([1, 2], [9, 8])
    >>> top_k_min_max(["apple","x","pi","atom"], 3, key=len)
    (['x', 'pi', 'atom'], ['apple', 'atom', 'pi'])
    """
    if k <= 0:
        return ([], [])
    smallest = heapq.nsmallest(k, iterable, key=key)
    largest = heapq.nlargest(k, iterable, key=key)
    return (smallest, largest)


In [9]:
# Tests — do not modify
a, b = top_k_min_max([5,1,9,2,8], 2)
assert a == [1,2] and b == [9,8]
s, l = top_k_min_max(["apple","x","pi","atom"], 3, key=len)
assert s == ['x','pi','atom'] and l == ['apple','atom','pi']
assert top_k_min_max([1,2,3], 0) == ([], [])
print("✅ Problem 4 tests passed.")


✅ Problem 4 tests passed.


## Problem 5 — Sort numbers by distance to a pivot (then by value)

Implement `sort_by_distance(nums, pivot)` that sorts numbers by `abs(x - pivot)` (nearest first). For equal distances, break ties by the **actual number** ascending.

Return a new `list` without mutating the input.


In [10]:
def sort_by_distance(nums: Iterable[float], pivot: float) -> List[float]:
    """Sort numbers by distance to pivot, then by numeric value.

    Examples
    --------
    >>> sort_by_distance([10, 5, 9, 11, 4], pivot=9)
    [9, 10, 8, 11, 4]
    """
    # NOTE: the example above is illustrative; actual output depends on input list.
    # YOUR CODE HERE
    return sorted(nums, key=lambda x: (abs(x - pivot), x))


In [11]:
# Tests — do not modify
data = [12, 8, 10, 9, 11, 7]
assert sort_by_distance(data, 10) == [10, 9, 11, 8, 12, 7]
assert sort_by_distance([3,1,5], 4) == [3,5,1]  # ascending tie-break on equal distance
print("✅ Problem 5 tests passed.")


✅ Problem 5 tests passed.


## Problem 6 — Multi-criteria sort for strings

Implement `smart_string_sort(strings)` that sorts by the following key tuple:

1. **Length** (shorter first)
2. **Case-insensitive** (via `.casefold()`)
3. **Original string** (to break ties deterministically)

Return a new list; do not mutate the input.


In [12]:
def smart_string_sort(strings: Iterable[str]) -> List[str]:
    """Sort by length, then case-insensitive value, then original string.

    Examples
    --------
    >>> smart_string_sort(["Bee", "be", "ant", "Ants"]) 
    ['be', 'Bee', 'ant', 'Ants']
    """
    # YOUR CODE HERE
    return sorted(strings, key=lambda s: (len(s), s.casefold(), s))


In [13]:
# Tests — do not modify
arr = ["Bee", "be", "ant", "Ants", "a", "Z", "zz"]
# length ↑, then case-insensitive ↑, then original ↑
assert smart_string_sort(arr) == ['a', 'Z', 'be', 'zz', 'ant', 'Bee', 'Ants']
print("✅ Problem 6 tests passed.")

✅ Problem 6 tests passed.


## Problem 7 — Stable sort demonstration (don't reorder equals)

Given pairs `(group, value)`, we will sort by `group` only and verify that values retain their **original relative order** within each group.

Implement `stable_group_sort(pairs)` that returns the pairs sorted by `group` using a **stable** strategy with `sorted`.


In [14]:
from dataclasses import dataclass

@dataclass
class Pair:
    group: str
    value: int

def stable_group_sort(pairs: Iterable[Pair]) -> List[Pair]:
    """Sort pairs by group only; preserve original order within equal groups.

    Examples
    --------
    >>> p = [Pair('B', 2), Pair('A', 1), Pair('B', 3), Pair('A', 4)]
    >>> [ (x.group, x.value) for x in stable_group_sort(p) ]
    [('A', 1), ('A', 4), ('B', 2), ('B', 3)]
    """
    # YOUR CODE HERE
    return sorted(pairs, key=lambda p: p.group)


In [15]:
# Tests — do not modify
orig = [Pair('B', 2), Pair('A', 1), Pair('B', 3), Pair('A', 4), Pair('B', 5)]
res = stable_group_sort(orig)
# Within 'A' group, values should appear as 1,4 (original relative order)
assert [p.value for p in res if p.group == 'A'] == [1,4]
# Within 'B' group, values should appear as 2,3,5 (original relative order)
assert [p.value for p in res if p.group == 'B'] == [2,3,5]
print("✅ Problem 7 tests passed.")


✅ Problem 7 tests passed.


## Problem 8 — Min/Max on empty iterables with `default` and `key`

Implement `safe_min(iterable, *, key=None, default=... )` and `safe_max(...)` that behave like built-ins **with** the `default` feature for empty iterables. If the iterable is empty **and no default is provided**, raise `ValueError` (mirroring `min([])`).

Constraints:
- Stream-friendly (single pass, no list copy).
- Support `key` like built-ins.


In [16]:
def _extreme(iterable: Iterable[Any], *, key: Optional[Callable[[Any], Any]],
             default: Any, find_min: bool) -> Any:
    it = iter(iterable)
    try:
        first = next(it)
    except StopIteration:
        if default is ...:
            raise ValueError("empty iterable and no default provided")
        return default
    if key is None:
        best_item = first
        for x in it:
            if (x < best_item) if find_min else (x > best_item):
                best_item = x
        return best_item
    else:
        best_item = first
        best_key = key(first)
        for x in it:
            kx = key(x)
            if (kx < best_key) if find_min else (kx > best_key):
                best_key = kx
                best_item = x
        return best_item

def safe_min(iterable: Iterable[Any], *, key: Optional[Callable[[Any], Any]] = None,
             default: Any = ...) -> Any:
    return _extreme(iterable, key=key, default=default, find_min=True)

def safe_max(iterable: Iterable[Any], *, key: Optional[Callable[[Any], Any]] = None,
             default: Any = ...) -> Any:
    return _extreme(iterable, key=key, default=default, find_min=False)


In [17]:
# Tests — do not modify
assert safe_min([3,1,2]) == 1
assert safe_max([3,1,2]) == 3
assert safe_min([], default=0) == 0
assert safe_max([], default=-1) == -1
try:
    safe_min([])
    raise AssertionError("expected ValueError")
except ValueError:
    pass
assert safe_min(["aa","b","cccc"], key=len) == 'b'
assert safe_max(["aa","b","cccc"], key=len) == 'cccc'
print("✅ Problem 8 tests passed.")


✅ Problem 8 tests passed.
